In [185]:
# Import libraries

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer 
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as Pipeline_imb

from sklearn.metrics import classification_report                # Performance accuracy, f1-score
from sklearn.metrics import roc_auc_score                        # performance en utilisant la courbe ROC et le score AUC
from sklearn.metrics import log_loss                             # Perte logarithmique
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings('ignore')

# Chemins d'accès aux données
repDataConsolidees = '../../dataConsolidees.gouv/'

# Liste des fichiers usagers disponibles
!pwd
!ls {repDataConsolidees}*  -lrt
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)


/Users/stephane/Documents/Professionnel/Formation/Certificats/Datascientest/4 - Projet AccidentsRoutiers/scripts/1 - ExplorationDonnees
ls: -lrt: No such file or directory
../../dataConsolidees.gouv/agg_accidents.csv
../../dataConsolidees.gouv/agg_usagers.csv
../../dataConsolidees.gouv/test_set.csv
../../dataConsolidees.gouv/test_stat.csv
../../dataConsolidees.gouv/test_stat_agrege.csv
../../dataConsolidees.gouv/train_set.csv
../../dataConsolidees.gouv/usagers.csv


In [165]:
# ETAPE 1 - Chgt des données 
train = pd.read_csv (repDataConsolidees + "train_set.csv", dtype={'user_actp' : str}, ) 
train.describe()

# Suppression des enregistrements sans valeur cible
train = train[train.user_gravite != -1]

feature = train.drop("user_gravite", axis=1)
target = train['user_gravite']

test = pd.read_csv (repDataConsolidees + "test_set.csv", dtype={'user_actp' : str}, ) 
test.describe()

test_feature = test.drop("user_gravite", axis=1)
test_target = test['user_gravite']

# Chgt du résultat des tests statistique de l'étape précédente : ces infos serviront à définir à éliminer une variable sur un seuil ou à la conserver
test_stat = pd.read_csv(repDataConsolidees + "test_stat_agrege.csv")



In [30]:
# Génération d'un df de test des Fonctions de Transformation
testTrain = train.head(20)
save_testTrain = testTrain.copy()


### ETAPE 2 - FONCTIONS DE TRANSFORMATION

In [169]:
# Fonction de sélection des variables à conserver/éliminer des features
def keep_as_feature(df:pd.DataFrame, variable:str, threshold:float) -> bool :
    """ Retourne True/False selon que threshold >= df.coef pour df.variable = variable
        Cette fonction sera utilisée pour définir si une variable doit être conservée ou éliminée des features en fonction d'un seuil
        d'acceptation.

        ARGS :
        - df : dataframe contenant les colonnes 'variable' et 'coef'
        - variable : (str) nom de la variable à tester
        - threshold (float) : seuil devant être atteint : le test se fait en valeur absolue du coef obtenu lors du test

        RETURN :
        - si le seuil a été atteint en valeur absolue : True
        - sinon : False
    """
    
    if not (df[(df['variable'] == variable) & (np.abs(df['coef'] >= threshold))].empty):
        return True

    return False

In [167]:
class WdropColumns (BaseEstimator, TransformerMixin) :
    """ FONCTION DE TRANSFORMATION PERMETTANT DE SUPPRIMER TOUTES LES COLONNES DEFINIES DANS UNE LISTE
Parameters
----------
- cols_to_drop (list) : Liste de noms de colonnes à supprimer de df

Functions 
---------
fit, fit_transform, transform

    """
    def __init__ (self, cols_to_drop:list=[]) :
        self.cols_to_drop = cols_to_drop

    def fit (self, X:pd.DataFrame, y:list=None) :
        return self

    def transform (self, X:pd.DataFrame):
        return X.drop(self.cols_to_drop, axis=1, errors='ignore')

# Finalement inutile du fait de l'utilisation de make_column_transformer qui filtre les colonnes non explicitement identifiées à traiter

In [110]:
# TRANSFORMER EN CHARGE DE GENERER LA DATE_ACCIDENT à partir de 'carac_an', 'carac_mois' et 'carac_jour'
class WdateAccident (BaseEstimator, TransformerMixin) :
    """ FONCTION DE TRANSFORMATION UTILISABLE DANS UN PIPELINE :
        - Génère une variable 'date_accident' à partir de 'carac_an', 'carac_mois' et 'carac_jour'
        - Supprime les variables utilisées pour générer cette date

Parameters
----------
- dropC (bool) : indicateur de suppression de X des variables utilisées pour générer 'date_accident'

Functions 
---------
fit, fit_transform, transform

"""
    def __init__ (self) :
        pass
        
    def fit (self, X:pd.DataFrame, y:list=None) :
        return self
    
    def transform (self, X:pd.DataFrame):
        cols = ['carac_an', 'carac_mois', 'carac_jour']

        # Contrôle existance des variable an, mois, jour dans X
        for col in cols :
            if col not in X.columns :
                return None
                
        # Création de la date                     
        X['date_accident'] =  X.apply (self._composerDate, axis=1)

            
        return X[['date_accident']]
    
    def _composerDate(self, row) :
        jour_str = str(int(row['carac_jour'])).zfill(2)
        mois_str = str(int(row['carac_mois'])).zfill(2)
        année_str = str(int(row['carac_an']))
    
        return int(f"{année_str}{mois_str}{jour_str}")


In [112]:
# DATE_ACCIDENT : test Classe transformation
#testTrain = save_testTrain.copy()
#display(testTrain)

wdateAccident = WdateAccident()
date_accident = wdateAccident.fit_transform(testTrain)

display(date_accident)



,date_accident
0,20200307
1,20200307
2,20200307
3,20200307
4,20200307
5,20200307
6,20200307
7,20200305
8,20200305
9,20200306


In [100]:
# TEST WdropColumns
wdropColumns = WdropColumns(['id_accident', 'id_vehicule', 'num_veh', 
                                  'carac_gps_lat', 'carac_gps_long', 'lieu_larrout', 'lieu_lartpc', 'lieu_nbv',
                                  'lieu_prof', 'lieu_vosp',
                                  'vehi_occutc', 'user_actp', 'user_etatp', 'user_locp',
                                  'user_secu3', 'user_sexe', 'agg_catv_perso', 'agg_nb_tue_vehicule', 'agg_nb_total_conductrice',
                                  'agg_is_conducteur_vehicule', 'agg_nb_pieton_vehicule', 'agg_nb_blesse_grave_vehicule',
                                  'agg_nb_passager_vehicule', 'agg_nb_indemne_vehicule', 'agg_nb_blesse_leger_vehicule', 
                                  'agg_nb_total_velo', 'agg_nb_total_vsp', 'agg_nb_total_moto', 'agg_nb_total_vl', 'agg_nb_total_pl',
                                  'agg_nb_total_va', 'agg_nb_total_conducteur', 'agg_nb_total_pieton', 'agg_nb_total_passager'])
new_df = wdropColumns.fit_transform(testTrain)
new_df



,carac_an,carac_mois,carac_jour,carac_hrmn,carac_agg,carac_atm,carac_col,carac_com,carac_dept,carac_int,...,user_an_nais,user_catu,user_place,user_secu1,user_secu2,user_trajet,user_gravite,agg_is_conductrice_vehicule,agg_nb_total_vehicule,date_accident
0,2020,3,7,835,2,7,6,657.0,91,9,...,1997.0,1,1,8,0,1,0,0,1,20200307
1,2020,3,7,835,2,7,6,657.0,91,9,...,1967.0,3,10,0,-1,5,1,0,1,20200307
2,2020,3,7,1330,2,1,3,174.0,91,2,...,1985.0,1,1,1,0,-1,0,0,2,20200307
3,2020,3,7,1330,2,1,3,174.0,91,2,...,2014.0,2,4,8,0,-1,2,0,2,20200307
4,2020,3,7,1330,2,1,3,174.0,91,2,...,1963.0,1,1,1,0,-1,0,0,2,20200307
5,2020,3,7,1850,2,1,6,215.0,91,1,...,1991.0,1,1,1,0,5,0,0,1,20200307
6,2020,3,7,1850,2,1,6,215.0,91,1,...,1991.0,3,10,0,-1,-1,2,0,1,20200307
7,2020,3,5,700,1,2,3,114.0,76,2,...,2001.0,1,1,2,4,1,2,0,2,20200305
8,2020,3,5,700,1,2,3,114.0,76,2,...,1972.0,1,1,1,8,1,0,1,2,20200305
9,2020,3,6,2029,2,1,2,297.0,68,1,...,1989.0,1,1,8,8,-1,0,0,2,20200306


In [330]:
# TRANSFORMER EN CHARGE DE GERER LES DIFFERENTS HYPERPARAMETRE  POUR CHAQUE VARIABLE
class Transformer (BaseEstimator, TransformerMixin) :
    """ FONCTION DE TRANSFORMATION UTILISABLE DANS UN PIPELINE : VARIABLES CATEGORIELLE    
Parameters
----------
- columnName (str)    : Nom de la colonne à transformer
        
- nan_stratey : 
  -> None               : aucune opération
  -> 'drop' (str)       : dropna() 
  -> 'specific' (str)   : Les fonctions _missing_values_fit() et _missing_values_transform() seront utilisées : elles pourrront être 
                          surchargées dans la classe mère
  -> strategy of SimpleImputer : application de la stratégie prise en charge par SimpleImputer ('constant', 'mean', 'most_frequent')
- fill_value :
  -> valeur à privilégier si nan_strategy='constant'

- encoder : 
  ->  None              : aucun encodage
  -> 'OneHotEncoder'    : encodage avec OneHotEncoder
  -> 'dummies'          : encodage avec get_dummies

- scaler :
  -> 'StandardScaler'   : Normalisation
  -> 'MinMaxScaler'     : Sérialisation (pour distrib. loi normale)

- dropC  (bool)         : Suppression de la colonne   

- outliers (bool)       : Reprise des Outliers    
        
Functions 
---------
- fit, fit_transform, transform
- _outliers_fit(X, y)           : Fonction de reprise des outliers lancée dans fit, cette fonction pourra être surchargée dans la classe fille
- _outliers_transform(X) -> X   : Fonction de reprise des outliers lancée dans transform, cette fonction sera surchargée dans la classe fille
- _missing_values_fit (X, y)    : Fonction de reprise des missing values lancée dans fit, cette fonction devra être surchargée et contiendra les règles spécifique.
- _missing_values_transfor(X) -> X : Fonction qui pourra être surchargée afin d'appliquer une règle spécifique de gestion des NaN                                
            
    """
    # Initialisation des constantes 
    OneHotEncoder = 'OneHotEncoder'             # Encoder
    dummies = 'dummies'                         # Encoder
    MinMaxScaler = 'MinMaxScaler'               # Scaler
    StandardScaler = 'StandardScaler'           # Scaler
    drop = 'drop'                               # NaN
    specific = 'specific'                       # NaN
    
    def __init__ (self, *, columnName:str, nan_strategy:str, encoder:str, scaler:str, dropC:bool, outliers:any, fill_value:any=None) :
        # Initialisation de la classe : les paramètres doivent être conservés en l'état
        self.columnName = columnName
        self.nan_strategy = nan_strategy              # missing value
        self.fill_value = fill_value    
        self.encoder = encoder                        # encoder
        self.scaler = scaler                          # scaler
        self.dropC = dropC                            # Conservarion de la colonne O/N
        self.outliers = outliers                      # Outlier

        self._initialized_encoder = None              # encoder
        self._dummies = False
        self._initialized_scaler = None               # scaler
        self._initialized_imputer = None              # imputer
        self._dropna = False                          # missing values
        self._specific_nan = False
    

    def _initialize_params(self):
        # Suppression de la colonne demandée, pas besoin d'initialiser les autres paramètres
        if self.dropC:
            return

        # Paramétrage missing_values
        if self.nan_strategy == Transformer.drop:
            self._dropna = True
            
        elif self.nan_strategy == Transformer.specific:
            self._specific_nan = True
            
        elif self.nan_strategy:
            self._initialized_imputer = SimpleImputer(strategy=self.nan_strategy, fill_value=self.fill_value)

        # Paramétrage encoder
        if self.encoder == Transformer.OneHotEncoder:
            self._initialized_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore', drop='first')
            
        elif self.encoder == Transformer.dummies:
            self._dummies = True

        # Paramétrage de Scaler
        if self.scaler == Transformer.StandardScaler:
            self._initialized_scaler = StandardScaler()
            
        elif self.scaler == Transformer.MinMaxScaler:
            self._initialized_scaler = MinMaxScaler()

 
    def fit (self, X:pd.DataFrame, y:list=None) :
        
        # Initialisation objets du trasnformer
        self._initialize_params()
        
        if self.dropC :
            return self                                     # demande de suppression de la colonne : rien à faire dans fit

        # outliers
        self._outliers_fit(X, y)

        # missing_values
        if self._initialized_imputer is not None :
            self._initialized_imputer.fit(X[[self.columnName]])
        
        elif self._specific_nan :
            # lancement de la fonction spécifique
            self._missing_values_fit(X, y)

        # One Hot Encoder
        if self._initialized_encoder is not None :
            self._initialized_encoder.fit(X[[self.columnName]])

        # scaler
        if self._initialized_scaler is not None :
            self._initialized_scaler.fit(X[[self.columnName]])

        return self
    
    def transform (self, X:pd.DataFrame):
        # Suppression de la colonne si demandé
        if self.dropC :
            return X.drop(self.columnName, axis=1)
            
        # Ouliers
        X = self._outliers_transform(X)

        # missing Values
        if self._initialized_imputer :
            # Traitement des NaN via Simple Imputer
            X[self.columnName] = self.simpleimputer.transform(X[[self.columnName]])
                    
        elif self._specific_nan :
            X = self._missing_values_transform(X)

        # OneHotEncoder
        if self._initialized_encoder is not None :
            encoded_array = self._initialized_encoder.transform(X[[self.columnName]])
            encoded_df = pd.DataFrame(encoded_array, columns=self._initialized_encoder.get_feature_names_out([self.columnName]))
            X = X.drop(self.columnName, axis=1)
            X = pd.concat([X.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

        elif self._dummies :
            X = pd.get_dummies(X, columns=[self.columnName], drop_first=True )
 
        # Scaler
        if self._initialized_scaler is not None :
            # Exécution du scaler sur la variable
            X[self.columnName] = self._initialized_scaler.transform(X[[self.columnName]])

        return X

    
    def _outliers_fit(self, X:pd.DataFrame, y=None) :
        """ Fonction de reprise des outliers qui sera exécutée dans le fit afin de calculer des données nécessaire à 
        _outliers_transform().

        Cette fonction ne fait rien mais si nécessaire, elle pourra être surchargée dans la classe fille.
        """
        
        return self

    
    def _outliers_transform(self, X:pd.DataFrame) -> pd.DataFrame :
        """ Fonction de reprise des outliers : cette fonction sera surchargée dans la classe fille. """
        if self.outliers is not None and self.outliers != False : 
            print (">>> La fonction _outliers doit être surchargée. Traitement des Outliers ignoré !!!")
        
        return X

        
    def _missing_values_fit(self, X:pd.DataFrame, y=None) :
        """ Fonction exécutée dans fit et permettant de réaliser des calculs préparatoire à une opération de transform. 
        Cette fonction devra être surchargée dans la classe fille afin de répondre au besoin spécifique.
        Par exemple : Réaliser un remplacement de la classe la plus représentée par clé de répartition

        Il sera préférable de rechercher ces valeurs de remplacement dans la fonction fit pour éviter la fuite de données.        
        """

        return self

        
    def _missing_values_transform(self, X:pd.DataFrame) -> pd.DataFrame :
        """ Fonction exécutée dans le trasnform et réalisant la trasnformation de X attendue.
        Elle utilisera les valeurs de remplacment qui auront été préalablement calculées dans _missing_values_fit (si nécessaire).

        Cette fonction devra être surchargée dans la classe fille.
        """
        # fillna()
        if self._specific_nan :
            print (">>> La fonction _missing_values_transform (et si besoin fit()) doit être surchargée. Traitement spécifique des missing values ignoré !!!")
        
        return X



## ETAPE 3 - FONCTIONS DE TRANSFORMATION SUR VARIABLES CATEGORIELLES


In [331]:
# TRANSFORMER VARIABLE 'carac_an' - 'carac_mois' - 'carac_jour'
class Wcarac_an (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str=None, dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

class Wcarac_mois (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str=None, dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

class Wcarac_jour (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str=None, dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'carac_agg'
class Wcarac_agg (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str=None, dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'carac_atm'
class Wcarac_atm (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'carac_col'
class Wcarac_col (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'carac_com'
class Wcarac_com (Transformer) :
    def __init__ (self, *, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', scaler:str=None, dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=scaler, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'carac_dept'
class Wcarac_dept (Transformer) :
    def __init__ (self, *, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', scaler:str=None, dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=scaler, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'carac_int'
class Wcarac_int (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'carac_lum'
class Wcarac_lum (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'lieu_catr'
class Wlieu_catr (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'lieu_circ'
class Wlieu_circ (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'lieu_infra'
class Wlieu_infra (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'lieu_plan'
class Wlieu_plan (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'lieu_situ'
class Wlieu_situ (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'lieu_surf'
class Wlieu_surf (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'lieu_vosp'
class Wlieu_vosp (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'vehi_catv'
class Wvehi_catv (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'agg_catv_perso'
class Wagg_catv_perso (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)


# TRANSFORMER VARIABLE 'vehi_choc'
class Wvehi_choc (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'vehi_manv'
class Wvehi_manv (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'vehi_motor'
class Wvehi_motor (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'vehi_obs'
class Wvehi_obs (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'vehi_obsm'
class Wvehi_obsm (Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'vehi_senc'
class Wvehi_senc(Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'user_catu'
class Wuser_catu(Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'user_secu1'
class Wuser_secu1(Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'user_secu2'
class Wuser_secu2(Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'user_secu3'
class Wuser_secu3(Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)

# TRANSFORMER VARIABLE 'user_trajet'
class Wuser_trajet(Transformer) :
    def __init__ (self, nan_strategy=None, fill_value=None, encoder:str='OneHotEncoder', dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, encoder=encoder, scaler=None, dropC=dropC, outliers=False)






In [243]:
# Test d'un transformer unitairement
test = testTrain[['carac_com']]
wcarac_com = Wcarac_com(nan_strategy=None, fill_value=None, encoder='OneHotEncoder', dropC=False,  )
new = wcarac_com.fit_transform(test)

#new

encoder= OneHotEncoder de type= <class 'str'>
Récupération de encoder()


## ETAPE 4 - FONCTIONS DE TRANSFORMATION SUR VARIABLES QUANTITATIVE

In [332]:
# TRANSFORMER VARIABLE 'lieu_nbv'
class Wlieu_nbv(Transformer) :
    def __init__ (self, nan_strategy=None, fill_value:any=None, encoder:str=None, scaler:str='StandardScaler', outliers:bool=False, dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, fill_value=fill_value, encoder=encoder, scaler=scaler, dropC=dropC, outliers=outliers)

# TRANSFORMER VARIABLE 'user_an_nais'
class Wuser_an_nais(Transformer) :
    def __init__ (self, nan_strategy=None, fill_value:any=None, encoder:str=None, scaler:str='StandardScaler', outliers:bool=False, dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, fill_value=fill_value, encoder=encoder, scaler=scaler, dropC=dropC, outliers=outliers)

# TRANSFORMER VARIABLE 'agg_nb_total_conductrice'
class Wagg_nb_total_conductrice(Transformer) :
    def __init__ (self, nan_strategy=None, fill_value:any=None, encoder:str=None, scaler:str='StandardScaler', outliers:bool=False, dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, fill_value=fill_value, encoder=encoder, scaler=scaler, dropC=dropC, outliers=outliers)

# TRANSFORMER VARIABLE 'agg_nb_total_vehicule'
class Wagg_nb_total_vehicule(Transformer) :
    def __init__ (self, nan_strategy=None, fill_value:any=None, encoder:str=None, scaler:str='StandardScaler', outliers:bool=False, dropC:bool=False) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, fill_value=fill_value, encoder=encoder, scaler=scaler, dropC=dropC, outliers=outliers)

# TRANSFORMER VARIABLE 'lieu_vma'
class Wlieu_vma(Transformer) :
    def __init__ (self, nan_strategy=None, fill_value:any=None, encoder:str=None, scaler:str='StandardScaler', outliers:bool=True, dropC:bool=True) :
        super().__init__(columnName=self.__class__.__name__[1:], nan_strategy=nan_strategy, fill_value=fill_value, encoder=encoder, scaler=scaler, dropC=dropC, outliers=outliers)

    def _outliers_transform(self, X:pd.DataFrame) -> pd.DataFrame:
        if self.outliers :
            # Gestion des Ouliers pour cette variable
            # REPRISE 1 
            ##   - Les valeurs < 10 seront multipliées par 10
            ##   - Les valeurs > 130 seront divisées par 10
            X[self.columnName] = X[self.columnName].apply(lambda valeur : valeur*10 if (valeur > 0) and (valeur < 10) else valeur/10 if valeur > 130 else valeur)

            # REPRISE 2 -  VMA = 130km/h en dehors d'une autoroute
            ## Force à 110 hors agglomération (supposition d'une erreur de saisie de 1 caractère
            ## Force à 50 en agglomération
            X[self.columnName] = X.apply(lambda row : 50 if (row[self.columnName]==130) and (row['lieu_catr'] != 1) and (row['carac_agg'] == 2) else row[self.columnName], axis=1)
            
        return X

    def _missing_values_fit (self, X:pd.DataFrame, y) :
        """ Surcharge Fonction missing values exécutée dans le fit
        Cette version permet de calculer les modalités les plus fréquentes selon la clé 'catr' + 'agg' pour les lignes <> -1.
        Ces valeurs seront utilisées dans _missing_value_transform() afin de remplacer les NaN.
        """
        self.mode_lieu_vma = X[X[self.columnName] != -1].groupby(['carac_agg', 'lieu_catr'])[self.columnName].agg(lambda x: x.value_counts().idxmax())

        return self
        
    def _missing_values_transform (self, X:pd.DataFrame) -> pd.DataFrame:
        """ Surcharge Fonction missing Values exécutée dans le transform """
        def replace_lieu_vma(row, mode_lieu_vma):
            if row[self.columnName] == -1:
                return mode_lieu_vma.get((row['carac_agg'], row['lieu_catr']), row[self.columnName])
    
            return row[self.columnName]
        
        X[self.columnName] = X.apply(lambda row: replace_lieu_vma(row, self.mode_lieu_vma), axis=1)

        return X




## ETAPE 5 - Construction d'un pipeline de pré processing

In [150]:
#  PREPARATION D'UN JEU DE DONNEES POUR TESTER LES PIPELINES
nb_enreg = 100000
x_train_pipeline = feature         #.head(nb_enreg)
y_train_pipeline = target          #.head(nb_enreg)

# HoldOut ==> périmètre de données utilisée pour la phase d'estimation du modèle
x_test_pipeline = test_feature     #.head(nb_enreg)
y_test_pipeline = test_target      #.head(nb_enreg)



In [ ]:
# Déclaration du pipeline preprocess

CaracPipeline_mct = make_column_transformer (
    (Wcarac_agg(), ['carac_agg']),
    (Wcarac_atm(), ['carac_atm']),
    (Wcarac_col(), ['carac_col']),
    (Wcarac_com(dropC=True), ['carac_com']),
    (Wcarac_dept(), ['carac_dept']),
    (Wcarac_int(), ['carac_int']),
    (Wcarac_lum(), ['carac_lum']),
    (Wcarac_an(), ['carac_an']),
    (Wcarac_mois(), ['carac_mois']),
    (Wcarac_jour(), ['carac_jour']),
    
)

LieuPipeline_mct = make_column_transformer (
    (Wlieu_catr(), ['lieu_catr']),
    (Wlieu_circ(), ['lieu_circ']),
    (Wlieu_infra(), ['lieu_infra']),
    (Wlieu_plan(), ['lieu_plan']),
    (Wlieu_situ(), ['lieu_situ']),
    (Wlieu_surf(), ['lieu_surf']),
    (Wlieu_vma(), ['lieu_vma']),
    (Wlieu_vosp(), ['lieu_vosp'])
)

VehiPipeline_mct = make_column_transformer (
    (Wvehi_catv(), ['vehi_catv']),
    (Wagg_catv_perso(), ['agg_catv_perso']),
    (Wvehi_choc(), ['vehi_choc']),
    (Wvehi_manv(), ['vehi_manv']),
    (Wvehi_motor(), ['vehi_motor']),
    (Wvehi_obs(), ['vehi_obs']),
    (Wvehi_obsm(), ['vehi_obsm']),
    (Wvehi_senc(), ['vehi_senc'])
)

UserPipeline_mct = make_column_transformer (
    (Wuser_catu(), ['user_catu']),
    (Wuser_secu1(), ['user_secu1']),
    (Wuser_secu2(), ['user_secu2']),
    (Wuser_secu3(), ['user_secu3']),
    (Wuser_trajet(), ['user_trajet']),
)

# Variables quantitatives
QuantitativePipeline_mct = make_column_transformer (
    (Wagg_nb_total_conductrice(), ['agg_nb_total_conductrice']),
    (Wagg_nb_total_vehicule(), ['agg_nb_total_vehicule']),
    (Wlieu_vma(), ['lieu_vma']),
    (Wlieu_nbv(), ['lieu_nbv']),
    (Wuser_an_nais(), ['user_an_nais'])
)

# Pipeline de création de la date au format integer : YYYYMMDD : la variable sera normalisée.
DateAccident = Pipeline(steps=[('date_accident', WdateAccident()),
                               ('Normalisation_date', StandardScaler())])

# Pipeline regroupant les 4 typologies
preprocess = ColumnTransformer(
    transformers=[
        ('DateAccident', DateAccident, ['carac_an', 'carac_mois', 'carac_jour']),
        ('QuantitativeVars', QuantitativePipeline_mct, ['agg_nb_total_conductrice', 'agg_nb_total_vehicule', 'lieu_vma', 'lieu_nbv', 'user_an_nais']),
        ('Caractéristique', CaracPipeline_mct, ['carac_agg', 'carac_atm', 'carac_col', 'carac_com', 'carac_dept', 'carac_int', 'carac_lum', 'carac_an', 'carac_mois', 'carac_jour']),
        ('Lieu', LieuPipeline_mct, ['lieu_catr', 'lieu_circ', 'lieu_infra', 'lieu_plan', 'lieu_situ', 'lieu_surf', 'lieu_vma', 'lieu_vosp']),
        ('Vehicule', VehiPipeline_mct, ['vehi_catv', 'agg_catv_perso', 'vehi_choc', 'vehi_manv', 'vehi_motor', 'vehi_obs', 'vehi_obsm', 'vehi_senc']),
        ('Usager', UserPipeline_mct, ['user_catu', 'user_secu1', 'user_secu2', 'user_secu3', 'user_trajet']),

    ], verbose=False
)
    

In [334]:
# Test du pipeline Preprocess
new = preprocess.fit_transform(x_train_pipeline)
print ("new de type: ", type(new))
new = pd.DataFrame(new)
#display(new)

new de type:  <class 'numpy.ndarray'>


## ETAPE 6 - PIPELINE FINAL : 
Enchainnement pipeline preprocessing + rééquilibrage + modele de regression Log.

In [154]:
#  PREPARATION D'UN JEU DE DONNEES POUR TESTER LES PIPELINES
nb_enreg = 100000
x_train_pipeline = feature          #.head(nb_enreg)
y_train_pipeline = target           #.head(nb_enreg)

x_test_pipeline = test_feature      #.head(nb_enreg)
y_test_pipeline = test_target       #.head(nb_enreg)


In [338]:
# TEST PIPELINE FINAL : PREPROCESSING + REEQUILIBRAGE + LANCEMENT MODELE DE RL
# à priori, il faut privilégier Pipeline de imblearn.pipeline

final_pipeline = Pipeline_imb(steps=[
    ('preprocessor', preprocess),
    ('under', RandomUnderSampler()),
    ('Selector', SelectKBest()),
    ('classifier', LogisticRegression(multi_class='ovr', solver='lbfgs'))       # Pourra être surchargé via GridSearch
], verbose=True)

# Test paramétrage de com/dept
params_com_dept = [
    {
        # Suppression des 2 colonnes
        'preprocessor__Caractéristique__wcarac_com__dropC' : [True],   
        'preprocessor__Caractéristique__wcarac_dept__dropC' : [True],   
        "Selector__k" : ['all']
   },
   {
        # Conservation que de com en encodage O/N
        'preprocessor__Caractéristique__wcarac_com__encoder' : [None, 'OneHotEncoder'],
        'preprocessor__Caractéristique__wcarac_com__scaler' : [None],
        'preprocessor__Caractéristique__wcarac_com__dropC' : [False],   
        'preprocessor__Caractéristique__wcarac_dept__dropC' : [True],   
        "Selector__k" : ['all']
   },
   {
        # Conservation que de dept en encodage O/N
        'preprocessor__Caractéristique__wcarac_com__dropC' : [True],   
        'preprocessor__Caractéristique__wcarac_dept__encoder' : [None, 'OneHotEncoder'],
        'preprocessor__Caractéristique__wcarac_dept__scaler' : [None],
        'preprocessor__Caractéristique__wcarac_dept__dropC' : [False],   
        "Selector__k" : ['all']
   },
   {
        # Conservation des 2 colonnes en encodage O/N
        'preprocessor__Caractéristique__wcarac_com__encoder' : [None, 'OneHotEncoder'],
        'preprocessor__Caractéristique__wcarac_com__scaler' : [None],
        'preprocessor__Caractéristique__wcarac_com__dropC' : [False],   
        'preprocessor__Caractéristique__wcarac_dept__encoder' : [None, 'OneHotEncoder'],
        'preprocessor__Caractéristique__wcarac_dept__scaler' : [None],
        'preprocessor__Caractéristique__wcarac_dept__dropC' : [False],   
        "Selector__k" : ['all']
   },
   {
        # Conservation des 2 colonnes en les considérant comme varaible quantitative et en les normalisant
        'preprocessor__Caractéristique__wcarac_com__encoder' : [None],
        'preprocessor__Caractéristique__wcarac_com__scaler' : ['StandardScaler'],
        'preprocessor__Caractéristique__wcarac_com__dropC' : [False],   
        'preprocessor__Caractéristique__wcarac_dept__encoder' : [None],
        'preprocessor__Caractéristique__wcarac_dept__scaler' : ['StandardScaler'],
        'preprocessor__Caractéristique__wcarac_dept__dropC' : [False],   
        "Selector__k" : ['all']
   }
]
grid = GridSearchCV(estimator=final_pipeline, param_grid=params_com_dept, cv=5)
grid.fit(x_train_pipeline, y_train_pipeline)
print ("Meilleurs paramètres : ", grid.best_params_)
print ("Avec un score de : ", grid.best_score_)


[Pipeline] ...... (step 1 of 4) Processing preprocessor, total=   3.5s
[Pipeline] ............. (step 2 of 4) Processing under, total=   0.1s
[Pipeline] .......... (step 3 of 4) Processing Selector, total=   0.1s
[Pipeline] ........ (step 4 of 4) Processing classifier, total=   1.8s
[Pipeline] ...... (step 1 of 4) Processing preprocessor, total=   4.0s
[Pipeline] ............. (step 2 of 4) Processing under, total=   0.1s
[Pipeline] .......... (step 3 of 4) Processing Selector, total=   0.1s
[Pipeline] ........ (step 4 of 4) Processing classifier, total=   2.0s
[Pipeline] ...... (step 1 of 4) Processing preprocessor, total=   3.4s
[Pipeline] ............. (step 2 of 4) Processing under, total=   0.1s
[Pipeline] .......... (step 3 of 4) Processing Selector, total=   0.1s
[Pipeline] ........ (step 4 of 4) Processing classifier, total=   1.7s
[Pipeline] ...... (step 1 of 4) Processing preprocessor, total=   3.6s
[Pipeline] ............. (step 2 of 4) Processing under, total=   0.1s
[Pipel

In [218]:
grid.cv_results_

{'mean_fit_time': array([5.58841581, 2.05875444]),
 'std_fit_time': array([0.32473237, 0.04837955]),
 'mean_score_time': array([1.00352225, 0.        ]),
 'std_score_time': array([0.01457031, 0.        ]),
 'param_Selector__k': masked_array(data=['all', 'all'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_preprocessor__Caractéristique__wcarac_com__dropC': masked_array(data=[False, False],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_preprocessor__Caractéristique__wcarac_com__encoder': masked_array(data=[None, 'OneHotEncoder'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_preprocessor__Caractéristique__wcarac_com__scaler': masked_array(data=[None, None],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_preprocessor__Caractéristique__wcarac_dept__dropC': masked_array(data=[True, True],
         

In [ ]:
# Test 1 - Logistic regression avec classifier multi_class='ovr'
params_modele = [
    {
        'classifier' : [LogisticRegression()],
        'classifier__C': [0.1, 1, 10],
        'classifier__solver': ['liblinear', 'lbfgs', 'sag', 'saga', 'newton-cg'],
        'classifier__multi_class': ['ovr']
    }]
grid = GridSearchCV(estimator=final_pipeline, param_grid=params, cv=3)
grid.fit(x_train_pipeline, y_train_pipeline)
print ("Meilleurs paramètres : ", grid.best_params_)
print ("Avec un score de : ", grid.best_score_)


[Pipeline] ...... (step 1 of 4) Processing preprocessor, total=   3.6s
[Pipeline] ............. (step 2 of 4) Processing under, total=   0.1s
[Pipeline] .......... (step 3 of 4) Processing Selector, total=   0.2s
[Pipeline] ........ (step 4 of 4) Processing classifier, total=   1.8s
[Pipeline] ...... (step 1 of 4) Processing preprocessor, total=   3.6s
[Pipeline] ............. (step 2 of 4) Processing under, total=   0.1s
[Pipeline] .......... (step 3 of 4) Processing Selector, total=   0.1s
[Pipeline] ........ (step 4 of 4) Processing classifier, total=   1.4s
[Pipeline] ...... (step 1 of 4) Processing preprocessor, total=   3.3s
[Pipeline] ............. (step 2 of 4) Processing under, total=   0.1s
[Pipeline] .......... (step 3 of 4) Processing Selector, total=   0.1s
[Pipeline] ........ (step 4 of 4) Processing classifier, total=   1.4s
[Pipeline] ...... (step 1 of 4) Processing preprocessor, total=   3.3s
[Pipeline] ............. (step 2 of 4) Processing under, total=   0.1s
[Pipel

In [ ]:
# Test 2 - Logistic regression avec classifier multi_class='multinominal'
params_modele = [
    {
        'classifier' : [LogisticRegression()],
        'classifier__C': [0.1, 1, 10],
        'classifier__solver': ['lbfgs', 'sag', 'saga', 'newton-cg'],
        'classifier__multi_class': ['multinomial']                        # incompatible avec 'liblinear'
    }]
grid = GridSearchCV(estimator=final_pipeline, param_grid=params, cv=3)
grid.fit(x_train_pipeline, y_train_pipeline)
print ("Meilleurs paramètres : ", grid.best_params_)
print ("Avec un score de : ", grid.best_score_)


In [ ]:
# Test 3 - SVC
params_modele = [
    {
        'classifier': [SVC()],
        'classifier__C': [0.1, 1, 10],
        'classifier__kernel': ['linear', 'rbf'],
        'classifier__gamma': ['scale', 'auto'],
    }]
grid = GridSearchCV(estimator=final_pipeline, param_grid=params, cv=3)
grid.fit(x_train_pipeline, y_train_pipeline)
print ("Meilleurs paramètres : ", grid.best_params_)
print ("Avec un score de : ", grid.best_score_)


In [ ]:
# Test 4 - Random Forest : sans les onehotencoder
params_modele = [
{
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [50, 100, 200],  # Nombre d'arbres dans la forêt
        'classifier__max_depth': [None, 10, 20, 30],  # Profondeur maximale de l'arbre
        'classifier__min_samples_split': [2, 5, 10],  # Nombre minimum d'échantillons nécessaires pour diviser un nœud
        'classifier__min_samples_leaf': [1, 2, 4],  # Nombre minimum d'échantillons nécessaires pour être à une feuille
    }
    ]
grid = GridSearchCV(estimator=final_pipeline, param_grid=params, cv=3)
grid.fit(x_train_pipeline, y_train_pipeline)
print ("Meilleurs paramètres : ", grid.best_params_)
print ("Avec un score de : ", grid.best_score_)


In [ ]:
# Recherche d'un modèle optimal pour notre projet (Test 1 + 2 + 3 + 4)
params_modele = [
    {
        'classifier' = [LogisticRegression()],
        'classifier__C': [0.1, 1, 10],
        'classifier__solver': ['liblinear', 'lbfgs', 'sag', 'saga', 'newton-cg'],
        'classifier__multi_class': ['ovr']
    },
    {
        'classifier' = [LogisticRegression()],
        'classifier__C': [0.1, 1, 10],
        'classifier__solver': ['lbfgs', 'sag', 'saga', 'newton-cg'],
        'classifier__multi_class': ['multinomial']                        # incompatible avec 'liblinear'
    },
    {
        'classifier': [SVC()],
        'classifier__C': [0.1, 1, 10],
        'classifier__kernel': ['linear', 'rbf'],
        'classifier__gamma': ['scale', 'auto'],
    },
    {
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [50, 100, 200],  # Nombre d'arbres dans la forêt
        'classifier__max_depth': [None, 10, 20, 30],  # Profondeur maximale de l'arbre
        'classifier__min_samples_split': [2, 5, 10],  # Nombre minimum d'échantillons nécessaires pour diviser un nœud
        'classifier__min_samples_leaf': [1, 2, 4],  # Nombre minimum d'échantillons nécessaires pour être à une feuille
    }
]
grid = GridSearchCV(estimator=final_pipeline, param_grid=params, cv=5)
grid.fit(x_train_pipeline, y_train_pipeline)
print ("Meilleurs paramètres : ", grid.best_params_)
print ("Avec un score de : ", grid.best_score_)


In [211]:
print ("Meilleurs paramètres : ", grid.best_params_)
print ("Avec un score de : ", grid.best_score_)

grid.cv_results_

Meilleurs paramètres :  {'Selector__k': 'all', 'preprocessor__Caractéristique__wcarac_com__dropC': True, 'preprocessor__Caractéristique__wcarac_com__encoder': None, 'preprocessor__Caractéristique__wcarac_com__scaler': None, 'preprocessor__Caractéristique__wcarac_dept__dropC': True, 'preprocessor__Caractéristique__wcarac_dept__encoder': None, 'preprocessor__Caractéristique__wcarac_dept__scaler': None}
Avec un score de :  0.5719108747170449


{'mean_fit_time': array([5.06763101, 1.92264471, 1.99051828, 1.93220181, 2.05006065,
        2.02154651, 5.46920123, 2.5165    , 2.966467  , 2.30294251,
        2.00474215, 2.11440482, 1.93944407, 1.95934124, 1.92948394,
        1.9386941 , 1.94482331, 1.92937298, 1.93698621, 1.97258706,
        1.92360015, 2.07399254, 2.35192156, 2.01501746, 1.98803229,
        2.05573778, 2.21489139, 1.95900221, 1.96478939, 2.05446253,
        2.18344288, 1.99874444, 2.08183084, 2.13172607, 2.09417634,
        2.00974078, 2.02497864, 2.20394721, 2.04270048, 2.06717167,
        2.10601339, 1.98666477, 2.06580272, 1.97222853, 2.09471059,
        2.08139877, 2.00175343, 1.96803622, 2.39870024, 2.39028029,
        1.99908819, 2.27958512, 2.23300428, 2.03858318, 2.03158517,
        2.38799887, 2.66342454, 2.40218315, 2.15303135, 1.93386035,
        1.9804872 , 2.22361803, 2.37444563, 2.07782459, 2.0931787 ,
        1.98893199, 2.02754354, 2.09201035, 2.07659602, 2.40019679,
        2.692137  , 2.10665746,

In [ ]:
# Lancement sans gridSearch

# Phase apprentissage

# Phase d'apprentissage sur train
final_pipeline.fit(x_train_pipeline, y_train_pipeline)

# Résultats sur train
y_train_pred = final_pipeline.predict(x_train_pipeline)
accuracy_train = accuracy_score(y_train_pipeline, y_train_pred)
cm_train = pd.crosstab(y_train_pipeline, y_train_pred, rownames=['Réelles'], colnames=['Prédites'])
report_train = classification_report(y_train_pipeline, y_train_pred)

print(f"Accuracy sur train : {accuracy_train}")
print(f"Confusion Matrix sur train :\n")
display(cm_train)
print(f"Classification Report sur train :\n{report_train}")

# Estimation du modèle sur test
y_test_pred = final_pipeline.predict(x_test_pipeline)
accuracy_test = accuracy_score(y_test_pipeline, y_test_pred)
cm_test = pd.crosstab(y_test_pipeline, y_test_pred, rownames=['réelles'], colnames=['Prédites'])
report_test = classification_report(y_test_pipeline, y_test_pred)

y_test_prob = final_pipeline.predict_proba(x_test_pipeline)[:, 1]

print(f"Accuracy test : {accuracy_test}")
print(f"Confusion Matrix test :\n")
display(cm_test)
print(f"Classification Report test :\n{report_test}")


In [159]:
pd.DataFrame(y_pred).value_counts()

0
0    127931
1     66587
2     38479
3     37720
Name: count, dtype: int64

In [492]:
# Lancement Preprocessing pipeline
#preprocess = PreprocessingPipeline.fit_transform(train)